##### Copyright 2025 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Search re-ranking using Gemini embeddings

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Search_reranking_using_embeddings.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/>

This notebook demonstrates the use of embeddings to re-rank search results. This walkthrough will focus on the following objectives:



1.   Setting up your development environment and API access to use Gemini.
2.   Using Gemini's function calling support to access the Wikipedia API.
3.   Embedding content via Gemini API.
4.   Re-ranking the search results.


This is how you will implement search re-ranking:


1.   The user will make a search query.
2.   You will use Wikipedia API to return the relevant search results.
3.   The search results will be embedded and their relevance will be evaluated by calculating distance metrics like cosine similarity.
4.   The most relevant search result will be returned as the final answer.

> The non-source code materials in this notebook are licensed under Creative Commons - Attribution-ShareAlike CC-BY-SA 4.0, https://creativecommons.org/licenses/by-sa/4.0/legalcode.

## Setup


In [2]:
%pip install -q -U "google-genai>=1.0.0"
%pip install -q wikipedia

  Preparing metadata (setup.py) ... done


Note: The [`wikipedia` package](https://pypi.org/project/wikipedia/) notes that it was "designed for ease of use and simplicity, not for advanced use", and that production or heavy use should instead "use [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) or one of the other more advanced [Python MediaWiki API wrappers](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python)".

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [3]:
from google.colab import userdata
from google import genai
API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=API_KEY)

In [4]:
# Define the model to be used in this notebook
# Note: experimental models have more limited quota for API calls.

MODEL_ID="gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}
model=f"models/{MODEL_ID}"

## Function Calling with the Gemini API

As stated earlier, this tutorial uses Gemini's function calling support to access the Wikipedia API. Please refer to the [docs](https://ai.google.dev/docs/function_calling) to learn more about function calling.

### Step 1: Define the Search Function

To cater to the search engine needs, you will design this function in the following way:


*   For each search query, the search engine will use the `wikipedia.search` method to get relevant topics.
*   From the relevant topics, the engine will choose `n_topics(int)` top candidates and will use `gemini-2.0-flash` to extract relevant information from the page.
*   The engine will avoid duplicate entries by maintaining a search history.


In [5]:
from typing import List
import textwrap

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

# This is the actual function that would be called based on the model's suggestion
# Define the function with type hints and docstring
def wikipedia_search(search_queries: List[str]) -> List[str]:
  """Search wikipedia for each query and summarize relevant docs.
    Args:
        search_queries: The user query to search wikipedia.

    Returns:
        A list of relevant information from wikipedia based on the query.
  """
  n_topics=3
  search_history = set() # tracking search history
  search_urls = []

  summary_results = []

  for query in search_queries:
    print(f'Searching for "{query}"')
    search_terms = wikipedia.search(query)

    print(f"Related search terms: {search_terms[:n_topics]}")
    for search_term in search_terms[:n_topics]: # select first `n_topics` candidates
      if search_term in search_history: # check if the topic is already covered
        continue

      print(f'Fetching page: "{search_term}"')
      search_history.add(search_term) # add to search history

      try:
        # extract the relevant data by using `gemini-2.0-flash` model
        page = wikipedia.page(search_term, auto_suggest=False)
        url = page.url
        print(f"Information Source: {url}")
        search_urls.append(url)
        page = page.content

        response = client.models.generate_content(
            model=model,
            contents=textwrap.dedent(
                f"""
                Extract relevant information
                about user's query: {query}
                From this source:

                {page}

                Note: Do not summarize. Only Extract and return the relevant information
                """
            ),
        )

        urls = [url]

        if response.candidates[0].citation_metadata:
          extra_citations = response.candidates[0].citation_metadata.citation_sources
          extra_urls = [source.url for source in extra_citations]
          urls.extend(extra_urls)
          search_urls.extend(extra_urls)
          print("Additional citations:", response.candidates[0].citation_metadata.citation_sources)
        try:
          text = response.text
        except ValueError:
          pass
        else:
          summary_results.append(text + "\n\nBased on:\n  " + ',\n  '.join(urls))

      except DisambiguationError:
        print(f"""Results when searching for "{search_term}" (originally for "{query}")
        were ambiguous, hence skipping""")
        continue

      except PageError:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

      except:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue

  print(f"Information Sources:")
  for url in search_urls:
    print('    ', url)

  return summary_results

In [6]:
example = wikipedia_search(["What are LLMs?"])

Searching for "What are LLMs?"
Related search terms: ['Large language model', 'DeepSeek', 'Retrieval-augmented generation']
Fetching page: "Large language model"
Information Source: https://en.wikipedia.org/wiki/Large_language_model
Large language model did not match with any page id, hence skipping.
Fetching page: "DeepSeek"
Information Source: https://en.wikipedia.org/wiki/DeepSeek
Fetching page: "Retrieval-augmented generation"
Information Source: https://en.wikipedia.org/wiki/Retrieval-augmented_generation
Information Sources:
     https://en.wikipedia.org/wiki/Large_language_model
     https://en.wikipedia.org/wiki/DeepSeek
     https://en.wikipedia.org/wiki/Retrieval-augmented_generation


Here is what the search results look like:

In [7]:
from IPython.display import display, Markdown

for e in example:
  display(Markdown(e))

LLMs are large language models. DeepSeek is a Chinese artificial intelligence company that develops LLMs. DeepSeek-R1 provides responses comparable to other contemporary large language models, such as OpenAI's GPT-4 and o1. Its training cost is reported to be significantly lower than other LLMs.


Based on:
  https://en.wikipedia.org/wiki/DeepSeek

According to the provided document:

*   LLMs are large language models.
*   RAG improves large language models (LLMs) by incorporating information retrieval before generating responses. Unlike traditional LLMs that rely on static training data, RAG pulls relevant text from databases, uploaded documents, or web sources.
*   Retrieval-Augmented Generation (RAG) is a method that allows large language models (LLMs) to retrieve and incorporate additional information before generating responses.
*   RAG enhances large language models (LLMs) by incorporating an information-retrieval mechanism that allows models to access and utilize additional data beyond their original training set.
*   RAG is not a complete solution to the problem of hallucinations in LLMs.
*   While RAG improves the accuracy of large language models (LLMs), it does not eliminate all challenges.


Based on:
  https://en.wikipedia.org/wiki/Retrieval-augmented_generation

### Step 2: Pass the System Instructions and Tools to the Model

### Define the prompt

In order to have multiple supporting search queries to the user's original query, you will instruct the model to generate more such queries. This would help the engine to cover the asked question on comprehensive levels.

In [8]:
instructions = """
    You have access to the Wikipedia API which you will be using
    to answer a user's query. Your job is to generate a list of search queries which
    might answer a user's question. Be creative by using various key-phrases from
    the user's query. To generate variety of queries, ask questions which are
    related to the user's query that might help to find the answer. The more
    queries you generate the better are the odds of you finding the correct answer.
    Here is an example:

    user: Tell me about Cricket World cup 2023 winners.

    function_call: wikipedia_search([
        'What is the name of the team that won the Cricket World Cup 2023?',
        'Who was the captain of the Cricket World Cup 2023 winning team?',
        'Which country hosted the Cricket World Cup 2023?',
        'What was the venue of the Cricket World Cup 2023 final match?',
        'Cricket World cup 2023',
        'Who lifted the Cricket World Cup 2023 trophy?'
    ])

    The search function will return a list of article summaries, use these to
    answer the  user's question.
"""

When using the Python SDK, you can provide Python functions directly as tools. The SDK automatically converts the Python function to declarations, handles the function call execution and response cycle for you. The Python SDK then automatically:

- Detects function call responses from the model.
- Call the corresponding Python function in your code.
- Sends the function response back to the model.
- Returns the model's final text response.

To use this, define your function with type hints and a docstring as we did above, and then pass the function itself (not a JSON declaration) as a tool:

Note: This approach only handles annotations of `AllowedType = (int | float | bool | str | list['AllowedType'] | dict[str, AllowedType])`

In order to yield creative and a more random variety of questions, you could also utilize the model's temperature parameter. Values can range from [0.0,1.0], inclusive. A value closer to 1.0 will produce responses that are more varied and creative, while a value closer to 0.0 will typically result in more straightforward responses from the model.

In [9]:
from google.genai import types

config = types.GenerateContentConfig(
    tools=[wikipedia_search],
    system_instruction=instructions,
) # Pass the function itself

### Step 3: Enable Automatic Function Calling and Call the API (Python SDK Only)

Now start a new chat with default value of `automatic_function_calling`. Using the default `disable=None`, the `genai.ChatSession` will handle the back and forth required to call the function, and return the final response:

In [10]:
chat = client.chats.create(model=model, config=config, history=[])

query = "Explain how deep-sea life survives."

response = chat.send_message(query)

Searching for "deep-sea life adaptations"
Related search terms: ['Deep sea', 'Deep-sea fish', 'Deep-sea gigantism']
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Deep sea did not match with any page id, hence skipping.
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Deep-sea fish did not match with any page id, hence skipping.
Fetching page: "Deep-sea gigantism"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_gigantism
Searching for "deep-sea organisms survival strategies"
Related search terms: ['Brine pool', 'List of longest-living organisms', 'Microorganism']
Fetching page: "Brine pool"
Information Source: https://en.wikipedia.org/wiki/Brine_pool
Brine pool did not match with any page id, hence skipping.
Fetching page: "List of longest-living organisms"
Information Source: https://en.wikipedia.org/wiki/List_of_longest-living_organisms
Fetching page: "Microorganism"
Information Source: htt

In [11]:
display(Markdown(response.text))

Deep-sea life survives by adapting to the extreme conditions of the deep ocean, including high pressure, low or no light, cold temperatures, and limited food resources. Some key adaptations and survival strategies include:

*   **Adaptations to pressure:** Organisms have developed physiological adaptations to withstand immense pressure.
*   **Adaptations to temperature:** Some organisms have adapted to colder temperatures. Decreasing temperature leads to increased cell size and lifespan, delaying sexual maturity and resulting in larger body size.
*   **Food scarcity:** Larger body size improves foraging ability. Larger offspring have greater food reserves for drifting. Some organisms can gorge on food and survive long periods without it. Larger animals have more efficient metabolisms.
*   **Reduced predation pressure:** Predation is less frequent in deeper waters.


Check for additional citations:

In [12]:
response.candidates[0].citation_metadata or 'No citations found'

'No citations found'

### Step 4: Understand Chat History

That looks like it worked. You can go through the chat history to see the details of what was sent and received in the function calls:

In [13]:
import json
for content in chat.get_history():
    display(Markdown("### " + content.role + ":"))
    for part in content.parts:
        if part.text:
            display(Markdown(part.text))
        if part.function_call:
            print(part.function_call.name, json.dumps(part.function_call.args, indent=2))
        if part.function_response:
            for res in part.function_response.response['result']:
                display(Markdown(res))
    print("-" * 80)

### user:

Explain how deep-sea life survives.

--------------------------------------------------------------------------------


### model:

wikipedia_search {
  "search_queries": [
    "deep-sea life adaptations",
    "deep-sea organisms survival strategies",
    "deep-sea food web",
    "hydrothermal vent ecosystems",
    "challenges of deep-sea life",
    "deep-sea bioluminescence",
    "deep-sea pressure adaptation",
    "deep-sea temperature adaptation",
    "deep-sea oxygen levels"
  ]
}
--------------------------------------------------------------------------------


...

This document primarily focuses on the Deepsea Challenger submersible and its expeditions, not on the challenges of deep-sea life in general. However, some information can be extracted that touches on relevant issues:

*   **Extreme Pressure:** The document repeatedly emphasizes the immense pressure at the Challenger Deep (114 megapascals or 16,500 pounds per square inch), highlighting the need for specialized materials and engineering to withstand it. This implicitly underscores the primary challenge for life at those depths.

*   **Specialized Equipment & Technology:** The need for innovations like Isofloat syntactic foam, pressure-balanced oil-filled thrusters, LED lighting arrays, specialized cameras, and reliable communication cables. This shows that surviving in and studying the deep sea requires cutting-edge technology.

*   **Life Support Systems:** Mentions of life support systems aboard the Deepsea Challenger highlight the challenges of providing a habitable environment for humans at such depths.

*   **Limited Exploration:** The Deepsea Challenger's initial problems with cameras, life support, and thrusters during test dives illustrates the practical difficulties of exploring the deep sea.

*   **Scientific Observation:** Reference to the observation of anemones, jellyfish and habitats on the bottom shows the existence of life, but says nothing about the challenges they face.



Based on:
  https://en.wikipedia.org/wiki/Deepsea_Challenger

--------------------------------------------------------------------------------


### model:

Deep-sea life survives by adapting to the extreme conditions of the deep ocean, including high pressure, low or no light, cold temperatures, and limited food resources. Some key adaptations and survival strategies include:

*   **Adaptations to pressure:** Organisms have developed physiological adaptations to withstand immense pressure.
*   **Adaptations to temperature:** Some organisms have adapted to colder temperatures. Decreasing temperature leads to increased cell size and lifespan, delaying sexual maturity and resulting in larger body size.
*   **Food scarcity:** Larger body size improves foraging ability. Larger offspring have greater food reserves for drifting. Some organisms can gorge on food and survive long periods without it. Larger animals have more efficient metabolisms.
*   **Reduced predation pressure:** Predation is less frequent in deeper waters.


--------------------------------------------------------------------------------


In the chat history you can see all 4 steps:

1. User: Asks the question about the total number of mittens.
2. Model: Determines that the `wikipedia_search` is helpful and sends a FunctionCall request to the user.
3. User: The Chat session automatically executes the function (due to `_automatic_function_calling` is enabled by default) and sends back a FunctionResponse with the calculated result.
4. Model: Uses the function's output to formulate the final answer and presents it as a text response.

## [Optional] Manually Execute the Function Call

If you want to understand what happened behind the scenes, this section executes the `FunctionCall` manually to demonstrate.

In [16]:
config  = {
        "tools": [wikipedia_search],
        "automatic_function_calling": {"disable": True}, # for manual execution
    }

chat    = client.chats.create(model=model, config=config, history=[])
response  = chat.send_message(query)

Initially the model returns a `FunctionCall`:

In [17]:
fc = response.candidates[0].content.parts[0].function_call
print(json.dumps(fc.args, indent=2))

{
  "search_queries": [
    "deep sea life survival"
  ]
}


Call the function with generated arguments to get the results.

In [18]:
if fc.name == "wikipedia_search":
    summaries = wikipedia_search(**fc.args)
    display(Markdown("\n ### Function execution result:"))
    for text in summaries:
        display(Markdown(text))

Searching for "deep sea life survival"
Related search terms: ['Deep sea mining', 'Stranded Deep', 'Deep-sea community']
Fetching page: "Deep sea mining"
Information Source: https://en.wikipedia.org/wiki/Deep_sea_mining
Deep sea mining did not match with any page id, hence skipping.
Fetching page: "Stranded Deep"
Information Source: https://en.wikipedia.org/wiki/Stranded_Deep
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Information Sources:
     https://en.wikipedia.org/wiki/Deep_sea_mining
     https://en.wikipedia.org/wiki/Stranded_Deep
     https://en.wikipedia.org/wiki/Deep-sea_community



 ### Function execution result:

*   The game takes place in the Pacific Ocean, where a plane crash survivor must survive in a procedurally generated world.
*   Players explore islands, reefs, and ocean trenches.
*   The tropical ocean is full of life forms, including fish, sea turtles, rays, lionfish, sea snakes, crown-of-thorns starfish, swordfish, whales, and sharks.
*   Some sharks are harmless, while others, like great whites, hammerhead sharks, and goblin sharks, are life-threatening.
*   Sharks can attack the player or flip their raft.
*   There are also three mythical sea creatures: a megalodon shark, a giant squid named Lusca, and a giant eel named Abaia. These are the most challenging enemies.
*   Players must manage their health, hunger, thirst, and sleep.
*   Hydration sources are scarce, with coconut water being the most common.
*   Food can be obtained from coconuts, fishing, and rations.
*   Players can find equipment in sunken shipwrecks and islands.


Based on:
  https://en.wikipedia.org/wiki/Stranded_Deep

**Challenges to Survival:**

*   **Darkness:** Absence of sunlight requires reliance on sinking matter from above, chemosynthesis, or vertical migration.
*   **Hyperbaricity:** Extreme pressure is a major environmental factor. Creatures adapt with smaller size, gelatinous flesh, minimal skeletal structure, and elimination of excess cavities. Pressure increases by one atmosphere every ten meters.
*   **Temperature:** Generally cold and homogeneous below the thermocline, with constant temperatures over long periods. Hydrothermal vents offer the exception, with extreme temperature gradients.
*   **Nutrient Scarcity:** Limited food supply requires opportunistic feeding strategies and adaptations like rapid growth and dispersal mechanisms.

**Adaptations and Strategies for Survival:**

*   **Chemosynthesis:** Bacteria at hydrothermal vents and cold seeps convert chemicals into energy, forming the base of the food web in these ecosystems.
*   **Gelatinous Bodies and Reduced Skeletons:** Help withstand extreme pressure.
*   **Bioluminescence:** Used for camouflage, attracting prey, or attracting a mate.
*   **Vertical Migration:** Moving to shallower zones at night to feed, then returning to deeper waters during the day.
*   **Opportunistic Feeding:** Rapid growth and the ability to utilize transient food resources.
*   **Tolerance for Temperature Change:** Fish displaying considerable tolerances for temperature change while making daily migrations through the thermocline.

**Energy Sources:**

*   **Marine Snow:** Sinking particulate organic matter from the photic zone.
*   **Occasional Surface Blooms:** Sudden influx of nutrients from surface blooms.
*   **Whale Falls:** Carcasses of whales provide a large pulse of organic matter.
*   **Chemosynthesis:** Energy production from chemicals at hydrothermal vents and cold seeps.

**Zones and Depths:**

*   **Mesopelagic (200-1000m):** Also known as the twilight zone with limited light, organisms in this layer have large eyes to maximize the amount of light in the environment.
*   **Bathyal (1000-4000m):** Also known as the midnight zone, almost no light, cold temperatures of 4 °C (39 °F), a low level of dissolved oxygen, and a lack of sufficient nutrients.
*   **Abyssal (4000-6000m):** Perpetual darkness, inhabited by chemotrophs and pressure-resistant predators.
*   **Hadal (below 6000m):** Deepest trenches with immense pressure.



Based on:
  https://en.wikipedia.org/wiki/Deep-sea_community

Now send the `summaries` to the model.

In [19]:
# Create a function response part
function_response_part = types.Part.from_function_response(
    name=fc.name,
    response={"result": summaries},
)

response = chat.send_message(function_response_part)

display(Markdown(response.text))

Deep-sea life survives in a challenging environment with darkness, extreme pressure, cold temperatures, and nutrient scarcity. Here's a breakdown of their survival strategies:

*   **Challenges:**
    *   **Darkness:** No sunlight means organisms rely on other energy sources.
    *   **Pressure:** Extreme pressure requires special adaptations.
    *   **Temperature:** Consistently cold temperatures.
    *   **Nutrient Scarcity:** Limited food requires unique feeding strategies.
*   **Adaptations and Strategies:**
    *   **Chemosynthesis:** Bacteria convert chemicals into energy at hydrothermal vents and cold seeps, forming the base of the food web.
    *   **Body Adaptations:** Gelatinous bodies and reduced skeletons help withstand pressure.
    *   **Bioluminescence:** Used for camouflage, attracting prey, and finding mates.
    *   **Vertical Migration:** Moving to shallower waters to feed at night and returning to deeper waters during the day.
    *   **Opportunistic Feeding:** Rapid growth and ability to use transient food resources.
*   **Energy Sources:**
    *   **Marine Snow:** Sinking organic matter from upper ocean layers.
    *   **Surface Blooms:** Nutrients from surface blooms.
    *   **Whale Falls:** Whale carcasses provide a large amount of organic matter.
    *   **Chemosynthesis:** Energy from chemicals at hydrothermal vents and cold seeps.



## Re-ranking the search results

Helper function to embed the content:

In [20]:
# @title Helper function to embed the content
from tqdm.auto import tqdm
from google.genai import types

tqdm.pandas()

from google.api_core import retry
import numpy as np

def make_embed_text_fn(model):

    @retry.Retry(timeout=300.0)
    def embed_fn(texts: list[str]) -> list[list[float]]:
        # Set the task_type to SEMANTIC_SIMILARITY and embed the batch of texts
        embeddings = client.models.embed_content(
            model=model,
            contents=texts,
            config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY"),
        ).embeddings
        return np.array([embedding.values for embedding in embeddings])

    return embed_fn


def create_embeddings(content: list[str]) -> np.ndarray:
    MODEL_ID = "embedding-001" # @param ["embedding-001", "text-embedding-004","gemini-embedding-exp-03-07"] {"allow-input":true, isTemplate: true}
    model = f"models/{MODEL_ID}"
    embed_fn = make_embed_text_fn(model)

    batch_size = 100  # at most 100 requests can be in one batch
    all_embeddings = []

    # Loop over the texts in chunks of batch_size
    for i in tqdm(range(0, len(content), batch_size)):
        batch = content[i:i + batch_size]
        embeddings = embed_fn(batch)
        all_embeddings.extend(embeddings)

    return np.array(all_embeddings).reshape(len(all_embeddings), -1)

Please refer to the [embeddings guide](https://ai.google.dev/docs/embeddings_guide) for more information on embeddings.

Your next step is to define functions that you can use to calculate similarity scores between two embedding vectors. These scores will help you decide which embedding vector is the most relevant vector to the user's query.


You will now implement cosine similarity as your metric. Here returned embedding vectors will be of unit length and so their L1 norm (`np.linalg.norm()`) will be ~1. Hence, calculating cosine similarity is esentially same as calculating their dot product score.

In [21]:
def dot_product(a: np.ndarray, b: np.ndarray):
  return (a @ b.T)

### Similarity with user's query

Now it's time to find the most relevant search result returned by the Wikipedia API.

Use Gemini API to get embeddings for user's query and search results.

In [22]:
search_res = create_embeddings(summaries)
embedded_query = create_embeddings([query])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Calculate similarity score:

In [23]:
sim_value = dot_product(search_res, embedded_query)
sim_value

array([[0.66792859],
       [0.75340682]])

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [24]:
display(Markdown(summaries[np.argmax(sim_value)]))

**Challenges to Survival:**

*   **Darkness:** Absence of sunlight requires reliance on sinking matter from above, chemosynthesis, or vertical migration.
*   **Hyperbaricity:** Extreme pressure is a major environmental factor. Creatures adapt with smaller size, gelatinous flesh, minimal skeletal structure, and elimination of excess cavities. Pressure increases by one atmosphere every ten meters.
*   **Temperature:** Generally cold and homogeneous below the thermocline, with constant temperatures over long periods. Hydrothermal vents offer the exception, with extreme temperature gradients.
*   **Nutrient Scarcity:** Limited food supply requires opportunistic feeding strategies and adaptations like rapid growth and dispersal mechanisms.

**Adaptations and Strategies for Survival:**

*   **Chemosynthesis:** Bacteria at hydrothermal vents and cold seeps convert chemicals into energy, forming the base of the food web in these ecosystems.
*   **Gelatinous Bodies and Reduced Skeletons:** Help withstand extreme pressure.
*   **Bioluminescence:** Used for camouflage, attracting prey, or attracting a mate.
*   **Vertical Migration:** Moving to shallower zones at night to feed, then returning to deeper waters during the day.
*   **Opportunistic Feeding:** Rapid growth and the ability to utilize transient food resources.
*   **Tolerance for Temperature Change:** Fish displaying considerable tolerances for temperature change while making daily migrations through the thermocline.

**Energy Sources:**

*   **Marine Snow:** Sinking particulate organic matter from the photic zone.
*   **Occasional Surface Blooms:** Sudden influx of nutrients from surface blooms.
*   **Whale Falls:** Carcasses of whales provide a large pulse of organic matter.
*   **Chemosynthesis:** Energy production from chemicals at hydrothermal vents and cold seeps.

**Zones and Depths:**

*   **Mesopelagic (200-1000m):** Also known as the twilight zone with limited light, organisms in this layer have large eyes to maximize the amount of light in the environment.
*   **Bathyal (1000-4000m):** Also known as the midnight zone, almost no light, cold temperatures of 4 °C (39 °F), a low level of dissolved oxygen, and a lack of sufficient nutrients.
*   **Abyssal (4000-6000m):** Perpetual darkness, inhabited by chemotrophs and pressure-resistant predators.
*   **Hadal (below 6000m):** Deepest trenches with immense pressure.



Based on:
  https://en.wikipedia.org/wiki/Deep-sea_community

### Similarity with Hypothetical Document Embeddings (HyDE)

Drawing inspiration from [Gao et al](https://arxiv.org/abs/2212.10496) the objective here is to generate a template answer to the user's query using `gemini-2.0-flash`'s internal knowledge. This hypothetical answer will serve as a baseline to calculate relevance of all the search results.

In [25]:
res = client.models.generate_content(
    model=model,
    contents=f"""
        Generate a hypothetical answer
        to the user's query by using your own knowledge. Assume that you know everything
        about the said topic. Do not use factual information, instead use placeholders
        to complete your answer. Your answer should feel like it has been written by a human.

        query: {query}
    """,
)

display(Markdown(res.text))

Okay, so picture this: you're used to sunshine, warm breezes, and maybe a decent sandwich for lunch. Now, toss all that out the window. The deep sea is the opposite of everything you know! It's like another planet, a swirling, inky black void. So how in the name of Neptune does *anything* live down there?

Well, it all boils down to adaptability, my friend. These critters have evolved some truly bizarre and ingenious strategies to survive in such a harsh environment.

First, let's tackle the pressure. Imagine the weight of a [Placeholder Number] elephants stacked on your head! That's the kind of pressure some of these creatures endure. They've overcome this through internal adaptations, like having [Placeholder Biological Adaptation], or flexible skeletons made of [Placeholder Material]. Their cells are literally structured differently, allowing them to function normally under immense crushing forces.

Then there's the darkness. Photosynthesis? Forget about it. Sunlight penetrates only so far, so deep-sea life has to get its energy from other sources. Many rely on what's called "marine snow" – a constant shower of organic detritus raining down from the surface. It's basically leftover crumbs from the sunlit zone, kind of like the ocean's garbage disposal service but in a tasty (for some) form. Certain creatures, like the [Placeholder Deep Sea Creature], have developed special filtering mechanisms to efficiently capture these tiny particles.

Others are predators, of course! And in the dark, bioluminescence is king. This is where creatures generate their own light through chemical reactions, using it to attract prey, find mates, or even defend themselves. Think of the anglerfish with its glowing lure – a classic example of deep-sea ingenuity. They even have symbiotic relationships with [Placeholder Bioluminescent Bacteria] that do all the hard work! It's like having a living flashlight permanently attached.

But perhaps the most fascinating survival strategy is chemosynthesis. Near hydrothermal vents and cold seeps, volcanic activity and geological processes release chemicals like methane and hydrogen sulfide. Certain bacteria can use these chemicals to create energy, just like plants use sunlight. These chemosynthetic bacteria form the base of the food web in these unique ecosystems, supporting entire communities of bizarre creatures like [Placeholder Deep Sea Worm] and [Placeholder Chemosynthetic Shrimp].

So, to put it simply, deep-sea life thrives through a combination of pressure resistance, ingenious food-finding strategies like scavenging and predation utilizing bioluminescence, and the remarkable ability to harness chemical energy in the absence of sunlight. It’s a testament to the power of evolution and the sheer resilience of life, even in the most extreme environments. It's weird, it's wonderful, and it's a constant reminder that our planet holds secrets beyond our wildest imaginations. I wouldn’t mind taking a [Placeholder Deep Sea Submarine] down there myself one day!


Use Gemini API to get embeddings for the baseline answer and compare them with search results

In [26]:
hypothetical_ans = create_embeddings([res.text])

  0%|          | 0/1 [00:00<?, ?it/s]

Calculate similarity scores to rank the search results

In [27]:
sim_value = dot_product(search_res, hypothetical_ans)
sim_value

array([[0.69387301],
       [0.8186843 ]])

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [28]:
display(Markdown(summaries[np.argmax(sim_value)]))

**Challenges to Survival:**

*   **Darkness:** Absence of sunlight requires reliance on sinking matter from above, chemosynthesis, or vertical migration.
*   **Hyperbaricity:** Extreme pressure is a major environmental factor. Creatures adapt with smaller size, gelatinous flesh, minimal skeletal structure, and elimination of excess cavities. Pressure increases by one atmosphere every ten meters.
*   **Temperature:** Generally cold and homogeneous below the thermocline, with constant temperatures over long periods. Hydrothermal vents offer the exception, with extreme temperature gradients.
*   **Nutrient Scarcity:** Limited food supply requires opportunistic feeding strategies and adaptations like rapid growth and dispersal mechanisms.

**Adaptations and Strategies for Survival:**

*   **Chemosynthesis:** Bacteria at hydrothermal vents and cold seeps convert chemicals into energy, forming the base of the food web in these ecosystems.
*   **Gelatinous Bodies and Reduced Skeletons:** Help withstand extreme pressure.
*   **Bioluminescence:** Used for camouflage, attracting prey, or attracting a mate.
*   **Vertical Migration:** Moving to shallower zones at night to feed, then returning to deeper waters during the day.
*   **Opportunistic Feeding:** Rapid growth and the ability to utilize transient food resources.
*   **Tolerance for Temperature Change:** Fish displaying considerable tolerances for temperature change while making daily migrations through the thermocline.

**Energy Sources:**

*   **Marine Snow:** Sinking particulate organic matter from the photic zone.
*   **Occasional Surface Blooms:** Sudden influx of nutrients from surface blooms.
*   **Whale Falls:** Carcasses of whales provide a large pulse of organic matter.
*   **Chemosynthesis:** Energy production from chemicals at hydrothermal vents and cold seeps.

**Zones and Depths:**

*   **Mesopelagic (200-1000m):** Also known as the twilight zone with limited light, organisms in this layer have large eyes to maximize the amount of light in the environment.
*   **Bathyal (1000-4000m):** Also known as the midnight zone, almost no light, cold temperatures of 4 °C (39 °F), a low level of dissolved oxygen, and a lack of sufficient nutrients.
*   **Abyssal (4000-6000m):** Perpetual darkness, inhabited by chemotrophs and pressure-resistant predators.
*   **Hadal (below 6000m):** Deepest trenches with immense pressure.



Based on:
  https://en.wikipedia.org/wiki/Deep-sea_community

You have now created a search re-ranking engine using embeddings!

## Next steps

I hope you found this example helpful! Check out more examples in the [Cookbook](https://github.com/google-gemini/cookbook) to learn more.